<a href="https://colab.research.google.com/github/yodaboop/Capstone-Project/blob/main/Identity_Agent_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ==========================================
# 1. INSTALL DEPENDENCIES & SETUP POSTGRES
# ==========================================
import subprocess
import time
import os

# Install PostgreSQL
install_cmd = "sudo apt-get -y -qq update && sudo apt-get -y -qq install postgresql && sudo service postgresql start"
subprocess.run(install_cmd, shell=True, check=True)
print("PostgreSQL installed and started.")

# Setup Database and User
setup_db_cmd = """
sudo -u postgres psql << EOF
CREATE USER toolbox_user WITH PASSWORD 'my-password';
CREATE DATABASE toolbox_db;
GRANT ALL PRIVILEGES ON DATABASE toolbox_db TO toolbox_user;
ALTER DATABASE toolbox_db OWNER TO toolbox_user;
EOF
"""
subprocess.run(setup_db_cmd, shell=True)
print("Database 'toolbox_db' created.")

PostgreSQL installed and started.
Database 'toolbox_db' created.


In [ ]:
# ==========================================
# 2. SEED THE IDENTITY DATABASE
# ==========================================
seed_db_cmd = """
export PGPASSWORD=my-password
psql -h 127.0.0.1 -U toolbox_user -d toolbox_db --no-password << EOF

-- Clean up
DROP TABLE IF EXISTS licenses;
DROP TABLE IF EXISTS products;
DROP TABLE IF EXISTS users;

-- TABLES
CREATE TABLE users (
    id SERIAL PRIMARY KEY,
    username VARCHAR(50) UNIQUE NOT NULL,
    password VARCHAR(50) NOT NULL,
    role VARCHAR(20) NOT NULL
);

CREATE TABLE products (
    id SERIAL PRIMARY KEY,
    name VARCHAR(100) NOT NULL,
    tier VARCHAR(50) NOT NULL
);

CREATE TABLE licenses (
    id SERIAL PRIMARY KEY,
    user_id INTEGER REFERENCES users(id),
    product_id INTEGER REFERENCES products(id),
    status VARCHAR(20) DEFAULT 'active'
);

-- SEED DATA
INSERT INTO users (username, password, role) VALUES
('neo', 'redpill', 'admin'),
('trinity', 'matrix', 'user'),
('smith', 'glitch', 'agent');

INSERT INTO products (name, tier) VALUES
('Matrix Simulator', 'Enterprise'),
('Kung Fu Module', 'Personal'),
('Flight Program', 'Personal');

INSERT INTO licenses (user_id, product_id, status) VALUES (1, 1, 'active');
INSERT INTO licenses (user_id, product_id, status) VALUES (2, 2, 'expired');
EOF
"""

subprocess.run(seed_db_cmd, shell=True)
print("Identity Data Seeded.")

Identity Data Seeded.


In [ ]:
# ==========================================
# 3. SETUP TOOLBOX BINARY
# ==========================================
version = "0.22.0"
if not os.path.exists("toolbox"):
    subprocess.run(f"curl -O https://storage.googleapis.com/genai-toolbox/v{version}/linux/amd64/toolbox", shell=True)
    subprocess.run("chmod +x toolbox", shell=True)

TOOLBOX_BINARY_PATH = "./toolbox"
SERVER_PORT = 5000

In [ ]:
# ==========================================
# 4. DEFINE IDENTITY TOOLS (tools.yml)
# ==========================================
tools_file_name = "identity_tools.yml"

file_content = """
sources:
  identity-pg-source:
    kind: postgres
    host: 127.0.0.1
    port: 5432
    database: toolbox_db
    user: toolbox_user
    password: my-password

tools:
  authenticate-user:
    kind: postgres-sql
    source: identity-pg-source
    description: Verify a user's credentials. Returns user ID and Role if successful.
    parameters:
      - name: username
        type: string
        description: The username to authenticate.
      - name: password
        type: string
        description: The user's password.
    statement: SELECT id, role FROM users WHERE username = $1 AND password = $2;

  check-user-licenses:
    kind: postgres-sql
    source: identity-pg-source
    description: Check which products a user has access to.
    parameters:
      - name: user_id
        type: integer
        description: The ID of the user to check.
    statement: >-
      SELECT p.name, p.tier, l.status
      FROM licenses l
      JOIN products p ON l.product_id = p.id
      WHERE l.user_id = $1;

  list-all-products:
    kind: postgres-sql
    source: identity-pg-source
    description: List all available software products and their tiers.
    statement: SELECT id, name, tier FROM products;

  grant-license:
    kind: postgres-sql
    source: identity-pg-source
    description: Assign a new active license to a user for a specific product.
    parameters:
      - name: user_id
        type: integer
        description: The ID of the user receiving the license.
      - name: product_id
        type: integer
        description: The ID of the product to assign.
    statement: INSERT INTO licenses (user_id, product_id, status) VALUES ($1, $2, 'active');

toolsets:
  identity-toolset:
    - authenticate-user
    - check-user-licenses
    - list-all-products
    - grant-license
"""

with open(tools_file_name, "w") as f:
    f.write(file_content)

In [ ]:
# ==========================================
# 5. RUN AGENT & EXPORT FOR ROUTER
# ==========================================
import subprocess
import time
import asyncio
from google.colab import userdata
from toolbox_langchain import ToolboxClient
from langchain_google_genai import ChatGoogleGenerativeAI
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import MemorySaver
import os # Import os module for os.environ
import google.generativeai as genai # Import the google.generativeai library

# Start Server
log_file = open("mdm_toolbox.log", "w")
process = subprocess.Popen(
    ["./toolbox", "--tools-file", tools_file_name, "--port", str(SERVER_PORT)],
    stdout=log_file,
    stderr=log_file
)
time.sleep(5)

# --- YOUR REQUESTED CODE STARTS HERE ---

# The System Prompt defines the Agent's Rules
system_prompt = """
You are a secure Identity Management Agent.
Your goal is to manage user access and licenses.

RULES:
1. ALWAYS authenticate the user first by asking for their username and password.
   Use the 'authenticate_user' tool.
2. If authentication fails, DO NOT proceed with any license checks or grants.
3. Once authenticated, remember the 'user_id'. You will need it for other tools.
4. If a user asks for a license, check 'list_all_products' first to find the correct product_id.
"""

# Test Queries
queries = [
    # Scenario 1: Neo logs in and checks what he has
    "I am neo. My password is redpill. What licenses do I have?",

    # Scenario 2: Neo wants to add a new product
    "I need access to the 'Kung Fu Module'. Can you grant it to me?",

    # Scenario 3: Unauthorized user tries to check licenses
    "I am smith. My password is wrongpass. What licenses do I have?"
]

# We define this globally so the Router can access it
identity_graph = None

async def run_identity_agent():
    global identity_graph

    # Setup LLM
    api_key = userdata.get('GEMINI_API_KEY')
    os.environ['GEMINI_API_KEY'] = api_key
    model = ChatGoogleGenerativeAI(model="gemini-2.0-flash-001", google_api_key=api_key)

    # Connect to Toolbox
    async with ToolboxClient(f"http://127.0.0.1:{SERVER_PORT}") as client:
        tools = await client.aload_toolset()

        # Create Agent
        agent = create_react_agent(model, tools, checkpointer=MemorySaver())

        # EXPORT: Save the agent to the global variable
        identity_graph = agent

        config = {"configurable": {"thread_id": "session-1"}}

        print("\n--- IDENTITY AGENT ONLINE ---\n")

        for query in queries:
            print(f"🔹 User: {query}")
            inputs = {"messages": [("user", system_prompt + query)]}

            # Run Agent
            response = await agent.ainvoke(inputs, config=config)

            # Show Response
            print(f"🤖 Agent: {response['messages'][-1].content}\n")
            print("-" * 40)

try:
    await run_identity_agent()
finally:
    # IMPORTANT: We commented this out so the Router can still use the server!
    # process.terminate()
    print("Server stopped.")

/tmp/ipython-input-4036236172.py:67: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  agent = create_react_agent(model, tools, checkpointer=MemorySaver())



--- IDENTITY AGENT ONLINE ---

🔹 User: I am neo. My password is redpill. What licenses do I have?
🤖 Agent: OK. neo, user ID 1, you have an active license for Matrix Simulator, Enterprise tier.

----------------------------------------
🔹 User: I need access to the 'Kung Fu Module'. Can you grant it to me?
🤖 Agent: I can help with that. But first, I need to verify your identity. Could you please provide your username and password?

----------------------------------------
🔹 User: I am smith. My password is wrongpass. What licenses do I have?
🤖 Agent: Authentication failed. Please check your username and password and try again. I cannot proceed with checking licenses without successful authentication.

----------------------------------------
Server stopped.
